# Starbucks Capstone Challenge

## Business Understanding

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

## Data Understanding

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

## Gather

In [1]:
# import the necessary libraries
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
# Change default settings to allow seeing all of the data in a table.
pd.options.display.max_columns = 100
pd.options.display.max_rows = 200
pd.set_option('display.max_colwidth', 200)

## Assess

In [3]:
# This function identifies the percentage of missing values in a dataframe

def get_missing_pct(file_name):
    
    '''
    Inputs:  Enter the dataframe name that you want to assess for missing values.
    
    Outputs:  The function will output the variable names and percentage missing.
    
    '''
    
    percent_missing_filename = file_name.isnull().sum()*100 /len(file_name)
    missing_values_filename = pd.DataFrame({'column_name': file_name.columns,
                                 'percent_missing': percent_missing_filename})
    missing_values = missing_values_filename.sort_values('percent_missing', ascending=False)
    
    return missing_values

In [4]:
# View the contents of the portfolio dataframe (note:  it is only 10 rows in length)
portfolio.head(10)

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [5]:
# A more detailed look at the portfolio data
portfolio.describe()

,reward,difficulty,duration
count,10.000000,10.000000,10.000000
mean,4.200000,7.700000,6.500000
std,3.583915,5.831905,2.321398
min,0.000000,0.000000,3.000000
25%,2.000000,5.000000,5.000000
50%,4.000000,8.500000,7.000000
75%,5.000000,10.000000,7.000000
max,10.000000,20.000000,10.000000


In [6]:
# check for missing values
get_missing_pct(portfolio)

,column_name,percent_missing
reward,reward,0.0
channels,channels,0.0
difficulty,difficulty,0.0
duration,duration,0.0
offer_type,offer_type,0.0
id,id,0.0


In [7]:
# Look at the structure of the portfolio dataframe
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 608.0+ bytes


There are to be 10 rows in the portfolio dataframe.  They are displayed above.  The data structure appears to be correct. Difficulty (amount spent to achieve promotion) ranges from 0 to 20 dollars.  Offers appear to last between 3 and 7 days duration.  Portfolio is not missing any data.

#### Issues with the portfolio dataframe

* channels have multiple items.  We should break this into separate columns.<cr>
* Separate the offer types into individual columns
* Consider changing the portfolio ID so it is between 1 and 10 to make for better visibility
* Rename id to offer_id as it can be confused with the id in the profile dataframe


In [8]:
# View the contents of the portfolio dataframe (note:  it is only 10 rows in length)
profile.head(5)

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [9]:
# A more detailed look at the profile dataframe
profile.describe()

,age,became_member_on,income
count,17000.000000,1.700000e+04,14825.000000
mean,62.531412,2.016703e+07,65404.991568
std,26.738580,1.167750e+04,21598.299410
min,18.000000,2.013073e+07,30000.000000
25%,45.000000,2.016053e+07,49000.000000
50%,58.000000,2.017080e+07,64000.000000
75%,73.000000,2.017123e+07,80000.000000
max,118.000000,2.018073e+07,120000.000000


In [10]:
# check for missing values
get_missing_pct(profile)

,column_name,percent_missing
gender,gender,12.794118
income,income,12.794118
age,age,0.000000
id,id,0.000000
became_member_on,became_member_on,0.000000


In [11]:
# Look at the structure of the profile dataframe
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


There are 17,000 rows in the profile dataframe.  The maximum age is showing as 118.  It is likely an outlier.  Looking at the structure of the data, there appear to be some missing values for gender and income.

#### Issues with the profile dataframe

* Some participants appear to have an excessive age of 118.  Investigate further.<cr>
* became_member_on should be converted to datetime<cr>
* Some gender are showing as None <cr>
* We should consider simplifying id to be a number starting at 1<cr>
* Income has some NaNs.  Will need to consider how to treat these NaNs.
* became_member_on should be converted to datetime

In [12]:
# View the contents of the transcript dataframe (note:  it is only 10 rows in length)
transcript.head(5)

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [13]:
# A more detailed look at the transcript dataframe
transcript.describe()

,time
count,306534.000000
mean,366.382940
std,200.326314
min,0.000000
25%,186.000000
50%,408.000000
75%,528.000000
max,714.000000


In [14]:
# Look at the possible values for the event varible
transcript.event.value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

In [15]:
# Look at possible values for the value field
transcript.value.sample(20)

232400                                                 {'amount': 7.41}
253775                 {'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
295487                 {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
27447                                                 {'amount': 24.53}
264205                 {'offer id': '3f207df678b143eea3cee63160fa8bed'}
36948                                                 {'amount': 38.61}
123473                 {'offer id': '3f207df678b143eea3cee63160fa8bed'}
67309                  {'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
176491                                                {'amount': 13.49}
20478                                                 {'amount': 14.91}
225218    {'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d', 'reward': 5}
263910    {'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4', 'reward': 2}
190525                                                {'amount': 24.88}
129988                                                 {'amount'

The value field contains a dictionary.  The key identifies if offer id is present it also contains an amount.  The value is either an offer_id, mapping to portfolio or an actual amount value.  These need to be moved into separate columns.  Also, notice that offer id sometimes appears as offer_id.

In [16]:
# Look at the structure of the transcript dataframe
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [17]:
# check for missing values
get_missing_pct(transcript)

,column_name,percent_missing
person,person,0.0
event,event,0.0
value,value,0.0
time,time,0.0


There are 306,534 rows in the transcript dataframe.  There do not appear to be any missing values.  Time ranges between 0 and 714 minutes.

#### Issues with the transcript dataframe

* Need to clean the value column of the "{'offer id': " and "}"<cr>
* Some rows in the value column contain a combination of the offer id and reward amount.  These will need to be separated and cleaned<cr>
* The value field also contains some amounts (spend).  These will also need to be cleaned and placed into their own separate column.


## Data Preparation

### Quality Issues

In [150]:
# make a copy of the data to make the changes
transcript_copy = transcript.copy(deep=False)
portfolio_copy = portfolio.copy(deep=False)
profile_copy = profile.copy(deep=False)

## Clean

In [151]:
# Identify the potential keys in the value column

value_key = []
for index, row in transcript.iterrows():
    for i in row['value']:
        if i in value_key:
            continue
        else:
            value_key.append(i)

print(value_key)

['offer id', 'amount', 'offer_id', 'reward']


In [152]:
# Expand the channel to have multiple columns for each channel
channels = portfolio_copy['channels'].str.join(sep='*').str.get_dummies(sep='*')
offer_type = pd.get_dummies(portfolio_copy['offer_type'])
df = pd.concat([portfolio_copy, channels, offer_type], axis=1, sort=False)
  
# Remove the channels and offer_type variables
df = df.drop(['channels'], axis=1)
    
# Rename the columns
columns = ['id', 'difficulty', 'duration', 'offer_type', 'reward', 'email', 'mobile', 'social',
           'web', 'bogo', 'discount', 'informational']
portfolio_clean = df[columns]


### Test

In [153]:
# View the portfolio_clean dataframe
portfolio_clean

,id,difficulty,duration,offer_type,reward,email,mobile,social,web,bogo,discount,informational
0,ae264e3637204a6fb9bb56bc8210ddfd,10,7,bogo,10,1,1,1,0,1,0,0
1,4d5c57ea9a6940dd891ad53e9dbe8da0,10,5,bogo,10,1,1,1,1,1,0,0
2,3f207df678b143eea3cee63160fa8bed,0,4,informational,0,1,1,0,1,0,0,1
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,7,bogo,5,1,1,0,1,1,0,0
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,20,10,discount,5,1,0,0,1,0,1,0
5,2298d6c36e964ae4a3e7e9706d1fb8c2,7,7,discount,3,1,1,1,1,0,1,0
6,fafdcd668e3743c1bb461111dcafc2a4,10,10,discount,2,1,1,1,1,0,1,0
7,5a8bc65990b245e5a138643cd4eb9837,0,3,informational,0,1,1,1,0,0,0,1
8,f19421c1d4aa40978ebb69ca19b0e20d,5,5,bogo,5,1,1,1,1,1,0,0
9,2906b810c7d4411798c6938adc9daaa5,10,7,discount,2,1,1,0,1,0,1,0


In [154]:
# using the value field, move the keys to columns and values to the values in the individual colums 
transcript_copy = pd.concat([transcript_copy, transcript_copy['value'].apply(pd.Series)], axis=1)
transcript_copy.sample(10)

,person,event,value,time,offer id,amount,offer_id,reward
101247,08dc093548804971b6049d7a50a78f21,transaction,{'amount': 11.21},276,NaN,11.21,NaN,NaN
216025,71dcdf175893482cb9d00762e20939c0,offer viewed,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},504,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,NaN
240950,95f37a4a6f8b4bf6b9f91d62db4457dc,offer completed,"{'offer_id': '2298d6c36e964ae4a3e7e9706d1fb8c2', 'reward': 3}",558,NaN,NaN,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0
143667,0091d2b6a5ea4defaa8393e4e816db60,transaction,{'amount': 17.61},384,NaN,17.61,NaN,NaN
289627,b496625e7c30473e8883e5eccecc1127,offer viewed,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},642,ae264e3637204a6fb9bb56bc8210ddfd,NaN,NaN,NaN
85175,7b044f4a5a4d4a62a356f3bd4aae6c69,transaction,{'amount': 14.73},210,NaN,14.73,NaN,NaN
45138,5bd256d63385406b94de643992856231,transaction,{'amount': 13.01},114,NaN,13.01,NaN,NaN
142016,efff6930f36344bd83633a006a5a497f,transaction,{'amount': 31.25},378,NaN,31.25,NaN,NaN
171171,4be19dff9783459eaeaa990e7f569897,offer viewed,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},414,2298d6c36e964ae4a3e7e9706d1fb8c2,NaN,NaN,NaN
256747,9f6e1a67b0cf4a38964452db825f1a7e,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},576,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN


In [155]:
#clean up dataset to unify multiple offer_id columns into one column
transcript_copy['offer_id']=np.where(transcript_copy['offer_id'].isnull(),
                                    transcript_copy['offer id'],transcript_copy['offer_id'])



In [156]:
# sort the dataframe by person and offer_id in prepraration for using .ffill()
transcript_sorted =transcript_copy.sort_values(['person', 'offer_id'])
transcript_sorted.head(20)

,person,event,value,time,offer id,amount,offer_id,reward
247879,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},576,2906b810c7d4411798c6938adc9daaa5,NaN,2906b810c7d4411798c6938adc9daaa5,NaN
258884,0009655768c64bdeb2e877511632db8f,offer completed,"{'offer_id': '2906b810c7d4411798c6938adc9daaa5', 'reward': 2}",576,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,2.0
113605,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},336,3f207df678b143eea3cee63160fa8bed,NaN,3f207df678b143eea3cee63160fa8bed,NaN
139992,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '3f207df678b143eea3cee63160fa8bed'},372,3f207df678b143eea3cee63160fa8bed,NaN,3f207df678b143eea3cee63160fa8bed,NaN
55972,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168,5a8bc65990b245e5a138643cd4eb9837,NaN,5a8bc65990b245e5a138643cd4eb9837,NaN
77705,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},192,5a8bc65990b245e5a138643cd4eb9837,NaN,5a8bc65990b245e5a138643cd4eb9837,NaN
153401,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},408,f19421c1d4aa40978ebb69ca19b0e20d,NaN,f19421c1d4aa40978ebb69ca19b0e20d,NaN
168413,0009655768c64bdeb2e877511632db8f,offer completed,"{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d', 'reward': 5}",414,NaN,NaN,f19421c1d4aa40978ebb69ca19b0e20d,5.0
187554,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},456,f19421c1d4aa40978ebb69ca19b0e20d,NaN,f19421c1d4aa40978ebb69ca19b0e20d,NaN
204340,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},504,fafdcd668e3743c1bb461111dcafc2a4,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN


In [157]:
# fill in the missing offer_id values
transcript_sorted['offer_id'] = transcript_sorted['offer_id'].ffill()
transcript_sorted.head(20)

,person,event,value,time,offer id,amount,offer_id,reward
247879,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},576,2906b810c7d4411798c6938adc9daaa5,NaN,2906b810c7d4411798c6938adc9daaa5,NaN
258884,0009655768c64bdeb2e877511632db8f,offer completed,"{'offer_id': '2906b810c7d4411798c6938adc9daaa5', 'reward': 2}",576,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,2.0
113605,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},336,3f207df678b143eea3cee63160fa8bed,NaN,3f207df678b143eea3cee63160fa8bed,NaN
139992,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '3f207df678b143eea3cee63160fa8bed'},372,3f207df678b143eea3cee63160fa8bed,NaN,3f207df678b143eea3cee63160fa8bed,NaN
55972,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168,5a8bc65990b245e5a138643cd4eb9837,NaN,5a8bc65990b245e5a138643cd4eb9837,NaN
77705,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},192,5a8bc65990b245e5a138643cd4eb9837,NaN,5a8bc65990b245e5a138643cd4eb9837,NaN
153401,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},408,f19421c1d4aa40978ebb69ca19b0e20d,NaN,f19421c1d4aa40978ebb69ca19b0e20d,NaN
168413,0009655768c64bdeb2e877511632db8f,offer completed,"{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d', 'reward': 5}",414,NaN,NaN,f19421c1d4aa40978ebb69ca19b0e20d,5.0
187554,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},456,f19421c1d4aa40978ebb69ca19b0e20d,NaN,f19421c1d4aa40978ebb69ca19b0e20d,NaN
204340,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},504,fafdcd668e3743c1bb461111dcafc2a4,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN


In [158]:
#r resort the values
transcript_sorted =transcript_copy.sort_values(['person', 'offer_id', 'time'])
transcript_sorted.head(20)

,person,event,value,time,offer id,amount,offer_id,reward
247879,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},576,2906b810c7d4411798c6938adc9daaa5,NaN,2906b810c7d4411798c6938adc9daaa5,NaN
258884,0009655768c64bdeb2e877511632db8f,offer completed,"{'offer_id': '2906b810c7d4411798c6938adc9daaa5', 'reward': 2}",576,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,2.0
113605,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},336,3f207df678b143eea3cee63160fa8bed,NaN,3f207df678b143eea3cee63160fa8bed,NaN
139992,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '3f207df678b143eea3cee63160fa8bed'},372,3f207df678b143eea3cee63160fa8bed,NaN,3f207df678b143eea3cee63160fa8bed,NaN
55972,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168,5a8bc65990b245e5a138643cd4eb9837,NaN,5a8bc65990b245e5a138643cd4eb9837,NaN
77705,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},192,5a8bc65990b245e5a138643cd4eb9837,NaN,5a8bc65990b245e5a138643cd4eb9837,NaN
153401,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},408,f19421c1d4aa40978ebb69ca19b0e20d,NaN,f19421c1d4aa40978ebb69ca19b0e20d,NaN
168413,0009655768c64bdeb2e877511632db8f,offer completed,"{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d', 'reward': 5}",414,NaN,NaN,f19421c1d4aa40978ebb69ca19b0e20d,5.0
187554,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},456,f19421c1d4aa40978ebb69ca19b0e20d,NaN,f19421c1d4aa40978ebb69ca19b0e20d,NaN
204340,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},504,fafdcd668e3743c1bb461111dcafc2a4,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN


In [159]:
# drop unnecessary columns
transcript_copy.drop(['offer id', 'value'],axis=1,inplace=True)
transcript_copy.head()

,person,event,time,amount,offer_id,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,NaN,2906b810c7d4411798c6938adc9daaa5,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN


### Issue

Change portfolio id to numbers 1 through 10 and rename portfolio.id to portfolio.offer

In [160]:
def create_mapping_dictionary(series):
    
    '''
    Create a mapping of unique strings from a dataframe to integers.
    
    Arguments:
        series - the variable to be mapped to simple numbers
        
    Output:
        Mapping of unique numbers to the original value in the variable
    '''
    
    iss, isn = dict(), 1
    for s in series.unique():
        iss[s] = int(isn)
        isn += 1
    return iss


def map_var_to_number(series, map_dictionary):
    
    '''
    Performs the mapping of the original value to the variable to the new value
    
    Arguments:
        series - variable to be mapped
        map_dicts - the output of the create_map_dictionary
        
    Output:
        Column with new simplified numbering
    '''
    
    return series.map(map_dictionary)

### Test 

In [161]:
# use the functions above to simplify the offer_number
offer_mapping = create_mapping_dictionary(portfolio_clean.id)
portfolio_clean['offer_number'] = map_var_to_number(portfolio_clean.id, offer_mapping)

<ipython-input-161-9663f710e381>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  portfolio_clean['offer_number'] = map_var_to_number(portfolio_clean.id, offer_mapping)


In [162]:
portfolio_clean.drop(['id'], axis=1, inplace=True)

### Test

In [163]:
# verify the offer_number is simplified and that id is dropped
portfolio_clean

,difficulty,duration,offer_type,reward,email,mobile,social,web,bogo,discount,informational,offer_number
0,10,7,bogo,10,1,1,1,0,1,0,0,1
1,10,5,bogo,10,1,1,1,1,1,0,0,2
2,0,4,informational,0,1,1,0,1,0,0,1,3
3,5,7,bogo,5,1,1,0,1,1,0,0,4
4,20,10,discount,5,1,0,0,1,0,1,0,5
5,7,7,discount,3,1,1,1,1,0,1,0,6
6,10,10,discount,2,1,1,1,1,0,1,0,7
7,0,3,informational,0,1,1,1,0,0,0,1,8
8,5,5,bogo,5,1,1,1,1,1,0,0,9
9,10,7,discount,2,1,1,0,1,0,1,0,10


In [164]:
# create a variable converting duration to hours for comparison to transcript data
portfolio_clean['max_duration'] = portfolio_clean.duration*24

In [165]:
# verify duration was created
portfolio_clean.head()

,difficulty,duration,offer_type,reward,email,mobile,social,web,bogo,discount,informational,offer_number,max_duration
0,10,7,bogo,10,1,1,1,0,1,0,0,1,168
1,10,5,bogo,10,1,1,1,1,1,0,0,2,120
2,0,4,informational,0,1,1,0,1,0,0,1,3,96
3,5,7,bogo,5,1,1,0,1,1,0,0,4,168
4,20,10,discount,5,1,0,0,1,0,1,0,5,240


### Issue

Change the id in profile to a simple number from 1 to n and reanme it user_id

In [166]:
# use the functions above to simplify the user idn in the profile dataframe
user_mapping = create_mapping_dictionary(profile_copy.id)
profile_copy['user_id'] = map_var_to_number(profile_copy.id, user_mapping)

In [167]:
# drop the id field
profile_copy.drop(['id'], axis=1, inplace=True)

### Test

In [168]:
# verify that the id is simplified and is now the user_id field
profile_copy.head()

,gender,age,became_member_on,income,user_id
0,None,118,20170212,NaN,1
1,F,55,20170715,112000.0,2
2,None,118,20180712,NaN,3
3,F,75,20170509,100000.0,4
4,None,118,20170804,NaN,5


In [173]:
offer_mapping

{'ae264e3637204a6fb9bb56bc8210ddfd': 1,
 '4d5c57ea9a6940dd891ad53e9dbe8da0': 2,
 '3f207df678b143eea3cee63160fa8bed': 3,
 '9b98b8c7a33c4b65b9aebfe6a799e6d9': 4,
 '0b1e1539f2cc45b7b9fa7c272da2e1d7': 5,
 '2298d6c36e964ae4a3e7e9706d1fb8c2': 6,
 'fafdcd668e3743c1bb461111dcafc2a4': 7,
 '5a8bc65990b245e5a138643cd4eb9837': 8,
 'f19421c1d4aa40978ebb69ca19b0e20d': 9,
 '2906b810c7d4411798c6938adc9daaa5': 10}

### Issue

Replace person with a simple user_id number from 1 to n.

In [169]:
# simplify the user id in the transcript data
transcript_sorted['user_id'] = map_var_to_number(transcript_sorted.person, user_mapping)
transcript_sorted['offer_number'] = map_var_to_number(transcript_sorted.offer_id, offer_mapping)

# change theh offer number to int32 data type
transcript_sorted['offer_number'] = transcript_sorted['offer_number'].astype('int32')

### Test

In [171]:
# verify the simplification worked
transcript_sorted.head(10)

,person,event,value,time,offer id,amount,offer_id,reward,user_id,offer_number
247879,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},576,2906b810c7d4411798c6938adc9daaa5,NaN,2906b810c7d4411798c6938adc9daaa5,NaN,3794,10
258884,0009655768c64bdeb2e877511632db8f,offer completed,"{'offer_id': '2906b810c7d4411798c6938adc9daaa5', 'reward': 2}",576,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,2.0,3794,10
113605,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},336,3f207df678b143eea3cee63160fa8bed,NaN,3f207df678b143eea3cee63160fa8bed,NaN,3794,3
139992,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '3f207df678b143eea3cee63160fa8bed'},372,3f207df678b143eea3cee63160fa8bed,NaN,3f207df678b143eea3cee63160fa8bed,NaN,3794,3
55972,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168,5a8bc65990b245e5a138643cd4eb9837,NaN,5a8bc65990b245e5a138643cd4eb9837,NaN,3794,8
77705,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},192,5a8bc65990b245e5a138643cd4eb9837,NaN,5a8bc65990b245e5a138643cd4eb9837,NaN,3794,8
153401,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},408,f19421c1d4aa40978ebb69ca19b0e20d,NaN,f19421c1d4aa40978ebb69ca19b0e20d,NaN,3794,9
168413,0009655768c64bdeb2e877511632db8f,offer completed,"{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d', 'reward': 5}",414,NaN,NaN,f19421c1d4aa40978ebb69ca19b0e20d,5.0,3794,9
187554,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},456,f19421c1d4aa40978ebb69ca19b0e20d,NaN,f19421c1d4aa40978ebb69ca19b0e20d,NaN,3794,9
204340,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},504,fafdcd668e3743c1bb461111dcafc2a4,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN,3794,7


In [174]:
# drop fields no longer needed
transcript_sorted.drop(['person', 'value', 'offer_id', 'offer id'], axis=1, inplace=True)

In [176]:
# resort transcript_sorted and reset the index
transcript_sorted =transcript_sorted.sort_values(['user_id','offer_number', 'time']).reset_index()
transcript_sorted.head(20)

,level_0,index,event,time,amount,reward,user_id,offer_number
0,0,135224,transaction,360,0.35,NaN,1,0
1,1,167626,transaction,414,0.74,NaN,1,0
2,2,182544,transaction,444,1.89,NaN,1,0
3,3,218392,transaction,510,5.04,NaN,1,0
4,4,230411,transaction,534,0.38,NaN,1,0
5,5,237364,transaction,552,2.12,NaN,1,0
6,6,274519,transaction,606,4.61,NaN,1,0
7,7,283959,transaction,630,0.06,NaN,1,0
8,8,301915,transaction,696,5.21,NaN,1,0
9,9,110828,offer received,336,NaN,NaN,1,5


In [111]:
#clean up dataset to unify multiple offer_id columns into one column
transcript_sorted['amount']=np.where(transcript_sorted['amount'].isnull(),
                                    0,transcript_sorted['amount'])
transcript_sorted['reward']=np.where(transcript_sorted['reward'].isnull(),
                                    0,transcript_sorted['reward'])



In [112]:
# check for null values
transcript_sorted.isnull().sum()

index           0
event           0
time            0
amount          0
reward          0
user_id         0
offer_number    0
dtype: int64

In [177]:
# check for the different types of events
transcript_sorted.event.value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

In [114]:
# add extra column to flag the previous offer id
transcript_sorted['offer_id_previous'] = transcript_sorted.groupby(['user_id','offer_number'])['offer_number'].shift(periods=-1)

transcript_sorted['event_subsequent'] = transcript_sorted.groupby(['user_id','offer_number'])['event'].shift(periods=-1)

In [115]:
# identify events where offer received was before offer viewed
transcript_sorted['viewed_valid']=np.where(((transcript_sorted['event']=='offer received') &
                                            (transcript_sorted['event_subsequent']=='offer viewed')&
                                    (transcript_sorted['offer_number']==transcript_sorted['offer_id_previous'])),1,0)
# identify events where offer received was before offer viewed
transcript_sorted['trans_aft_view']=np.where(((transcript_sorted['event']=='offer viewed') &
                                              (transcript_sorted['event_subsequent']=='transaction')&
                                    (transcript_sorted['offer_number']==transcript_sorted['offer_id_previous'])),1,0)
# identify events where offer received was before offer viewed
transcript_sorted['comp_aft_view']=np.where(((transcript_sorted['event']=='offer viewed') &
                                             (transcript_sorted['event_subsequent']=='offer completed')&
                                    (transcript_sorted['offer_number']==transcript_sorted['offer_id_previous'])),1,0)
# identify events where offer received was before offer viewed
transcript_sorted['comp_aft_trans']=np.where(((transcript_sorted['event']=='transaction') &
                                              (transcript_sorted['event_subsequent']=='offer completed')&
                                    (transcript_sorted['offer_number']==transcript_sorted['offer_id_previous'])),1,0)

# identify events where offer received was before offer viewed
transcript_sorted['received']=np.where(((transcript_sorted['event']=='offer received')),1,0)
# identify events where offer received was before offer viewed
transcript_sorted['viewed']=np.where(((transcript_sorted['event']=='offer viewed')),1,0)
# identify events where offer received was before offer viewed
transcript_sorted['trans']=np.where(((transcript_sorted['event']=='transaction')),1,0)
# identify events where offer received was before offer viewed
transcript_sorted['completed']=np.where(((transcript_sorted['event']=='offer completed')),1,0)

In [116]:
# resort the transcript_sorted file and reset the index
transcript_sorted =transcript_sorted.sort_values(['user_id','offer_number', 'time']).reset_index()
transcript_sorted.head()

,level_0,index,event,time,amount,reward,user_id,offer_number,offer_id_previous,event_subsequent,viewed_valid,trans_aft_view,comp_aft_view,comp_aft_trans,received,viewed,trans,completed
0,0,135224,transaction,360,0.35,0.0,1,0,0.0,transaction,0,0,0,0,0,0,1,0
1,1,167626,transaction,414,0.74,0.0,1,0,0.0,transaction,0,0,0,0,0,0,1,0
2,2,182544,transaction,444,1.89,0.0,1,0,0.0,transaction,0,0,0,0,0,0,1,0
3,3,218392,transaction,510,5.04,0.0,1,0,0.0,transaction,0,0,0,0,0,0,1,0
4,4,230411,transaction,534,0.38,0.0,1,0,0.0,transaction,0,0,0,0,0,0,1,0


In [117]:
# summarize the rows of data for each user_id, offer_number so 1 row exists for each user_id and offer_number
trans_abbr_summary = transcript_sorted.groupby(['user_id', 'offer_number']).agg({'viewed_valid': ['max'],
                                                                            'trans_aft_view': ['max'],
                                                                             'comp_aft_view': ['max'],
                                                                             'comp_aft_trans': ['max'],
                                                                        'received': ['max'],
                                                                         'viewed': ['max'],
                                                                         'trans': ['max'],
                                                                         'completed': ['max']
                                                                        
                                                                        }).reset_index()

In [118]:
# check the results of the summarization
trans_abbr_summary.head(20)

,user_id,offer_number,viewed_valid,trans_aft_view,comp_aft_view,comp_aft_trans,received,viewed,trans,completed
,,,max,max,max,max,max,max,max,max
0,1,0,0,0,0,0,0,0,1,0
1,1,5,1,0,0,0,1,1,0,0
2,1,6,1,0,1,0,1,1,0,1
3,1,7,1,0,1,0,1,1,0,1
4,1,10,1,0,0,0,1,1,0,0
5,2,0,0,0,0,0,0,0,1,0
6,2,3,0,0,0,0,1,0,0,0
7,2,4,0,0,0,0,1,0,0,1
8,3,0,0,0,0,0,0,0,1,0


In [119]:
# merge transcript summary data with the cleaned portfolio data
transcript_merge = trans_abbr_summary.merge(portfolio_clean, on='offer_number', how='left')
transcript_merge.head()


C:\Users\dsbb1\AppData\Roaming\Python\Python38\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,offer_number,"(user_id, )","(offer_number, )","(viewed_valid, max)","(trans_aft_view, max)","(comp_aft_view, max)","(comp_aft_trans, max)","(received, max)","(viewed, max)","(trans, max)","(completed, max)",difficulty,duration,offer_type,reward,email,mobile,social,web,bogo,discount,informational,max_duration
0,0,1,0,0,0,0,0,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,1,5,1,0,0,0,1,1,0,0,20.0,10.0,discount,5.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,240.0
2,6,1,6,1,0,1,0,1,1,0,1,7.0,7.0,discount,3.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,168.0
3,7,1,7,1,0,1,0,1,1,0,1,10.0,10.0,discount,2.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,240.0
4,10,1,10,1,0,0,0,1,1,0,0,10.0,7.0,discount,2.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,168.0


In [120]:
# drop unnecessary fields
transcript_merge = transcript_merge.drop('offer_number', axis = 1)


In [121]:
# verify the merge
transcript_merge.head()

,"(user_id, )","(offer_number, )","(viewed_valid, max)","(trans_aft_view, max)","(comp_aft_view, max)","(comp_aft_trans, max)","(received, max)","(viewed, max)","(trans, max)","(completed, max)",difficulty,duration,offer_type,reward,email,mobile,social,web,bogo,discount,informational,max_duration
0,1,0,0,0,0,0,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5,1,0,0,0,1,1,0,0,20.0,10.0,discount,5.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,240.0
2,1,6,1,0,1,0,1,1,0,1,7.0,7.0,discount,3.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,168.0
3,1,7,1,0,1,0,1,1,0,1,10.0,10.0,discount,2.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,240.0
4,1,10,1,0,0,0,1,1,0,0,10.0,7.0,discount,2.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,168.0


In [122]:
# Rename the columns
columns = ['user_id', 'offer_number','viewed_valid', 'trans_aft_view', 'comp_aft_view',
                                       'comp_aft_trans', 'received', 'viewed', 'trans',
                                        'completed', 'difficulty', 'duration', 'offer_type', 'reward',
                                       'email', 'mobile', 'social', 'web', 'bogo', 'discount',
                                       'informational', 'max_duration']
transcript_merge.columns = [columns]
transcript_merge.head()

,user_id,offer_number,viewed_valid,trans_aft_view,comp_aft_view,comp_aft_trans,received,viewed,trans,completed,difficulty,duration,offer_type,reward,email,mobile,social,web,bogo,discount,informational,max_duration
0,1,0,0,0,0,0,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5,1,0,0,0,1,1,0,0,20.0,10.0,discount,5.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,240.0
2,1,6,1,0,1,0,1,1,0,1,7.0,7.0,discount,3.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,168.0
3,1,7,1,0,1,0,1,1,0,1,10.0,10.0,discount,2.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,240.0
4,1,10,1,0,0,0,1,1,0,0,10.0,7.0,discount,2.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,168.0


## Error ocurring here

In [124]:
# test to see if a the condition exists -  This is where the error is occurring
df = transcript_merge[(transcript_merge['viewed_valid']==1)&(transcript_merge['completed']==1)]
df.viewed_valid.sum()

viewed_valid    0.0
dtype: float64

In [ ]:
df.head()

In [ ]:
x = transcript_merge
df = x[(x.completed==1)]
df.completed.sum()

In [ ]:
df = transcript_merge

cond1 = df['viewed_valid'] == 1    
cond2 = df['completed'] == 1


x = df[cond1 & cond2]

In [ ]:
x.completed.sum()

In [ ]:
x.mean()

In [ ]:
transcript_merge.mean()

In [ ]:
# this defines group 1 for BOGO and Discounts
transcript_merge['group'] = np.where(((transcript_merge['bogo']==1) & (transcript_merge['received'] == 1) & (transcript_merge['viewed_valid'] == 1) & (transcript_merge['trans'] == 1) & (transcript_merge['completed'] == 1)),1,3)

In [ ]:
group1 = transcript_user_summary[[]]

In [ ]:
    #subset offer ids that have transactions or conversions by person and offer_id
    conversion_ids=df[['person','offer_id']][(df['event']=='transaction') | (df['event']=='offer completed') ].groupby(['person','offer_id']).count().reset_index()

    #check for unique person-offer_id pairs that consist of offers received 
    offers_received_only=df[['person','offer_id']][df['event']=='offer received'].groupby(['person','offer_id']).count().reset_index()

    #create merged dataset to diffrentiate groups
    check_merge=conversion_ids.merge(offers_received_only,how='right',on=['person','offer_id'],indicator=True)
    return check_merge


In [ ]:
alldata_balance = alldata[(alldata[IBRD] !=0) | (alldata[IMF] !=0)]



In [ ]:
data = transcript_merge[(transcript_merge['discount']==1) & (transcript_merge['received']==1) & 
                                    (transcript_merge['viewed_valid']==1) &
                                    (transcript_merge['trans']==1) & 
                                    (transcript_merge['completed']==1)]


In [ ]:
data.bogo.sum()

In [ ]:
# this defines group 1 for informational promos
transcript_user_summary['group'] = np.where((transcript_user_summary['offer_number'].isin([3,8])) &
                                    (transcript_user_summary['offer_received'] == 1) & 
                                    (transcript_user_summary['offer_viewed'] == 1) & 
                                    (transcript_user_summary['transaction'] == 1) &                                     
                                    (transcript_user_summary['offer_completed'] == 0),1,3)

In [ ]:
transcript_merge[[transcript_merge]transcript_user_summary['offer_number'].isin([3,8])) &
                                    (transcript_user_summary['offer_received'] == 1) & 
                                    (transcript_user_summary['offer_viewed'] == 1) & 
                                    (transcript_user_summary['transaction'] == 1) &                                     
                                    (transcript_user_summary['offer_completed'] == 0]

In [ ]:
# this defines group 1 for informational promos
transcript_merge['group'] = np.where(
                                     transcript_merge['bogo']== 1 &
                                     transcript_merge['received'] == 1 & 
                                     transcript_merge['viewed_valid'] == 1 & 
                                     transcript_merge['trans'] == 1 &                                     
                                     transcript_merge['completed'] == 1,1,3)

In [ ]:
transcript_sorted

In [ ]:
def create_steps(df, text, col_name):
    text_to_search = text
    new_col_name = col_name
    df[new_col_name] = df.event.apply(lambda x: 1 if x == text_to_search else 0)

In [ ]:
create_steps(transcript_sorted, 'offer received', 'offer_received')
create_steps(transcript_sorted, 'offer viewed', 'offer_viewed')
create_steps(transcript_sorted, 'transaction', 'transaction')
create_steps(transcript_sorted, 'offer completed', 'offer_completed')

In [ ]:
transcript_sorted.head(20)

In [ ]:
transcript_user_summary = transcript_sorted.groupby(['user_id', 'offer_number']).agg({'offer_received': ['max'],
                                                                            'offer_viewed': ['max'],
                                                                             'transaction': ['sum'],
                                                                             'reward': ['sum'],'amount': ['sum'],
                                                                                      
                                                                             'offer_completed': ['max']}).reset_index()

### Test

In [ ]:
transcript_user_summary.sample(20)

In [ ]:
transcript_user_summary.info()

In [ ]:
transcript_user_summary.rename(columns={'offer_received, max':'offer_received',
                                  'offer_viewed, max': 'offer_viewed',
                                  'offer_completed, max': 'offer_completed'},inplace=True)

In [ ]:
portfolio_clean.sample()

In [ ]:
transcript_user_summary.describe()

In [ ]:
transcript_user_summary1 = transcript_user_summary.to_numpy()

In [ ]:
# this defines group 1 for informational promos
transcript_user_summary['group'] = np.where((transcript_user_summary['offer_number'].isin([3,8])) &
                                    (transcript_user_summary['offer_received'] == 1) & 
                                    (transcript_user_summary['offer_viewed'] == 1) & 
                                    (transcript_user_summary['transaction'] == 1) &                                     
                                    (transcript_user_summary['offer_completed'] == 0),1,3)

In [ ]:
# this defines group 1 for BOGO and Discounts
transcript_user_summary['group'] = np.where((transcript_user_summary['offer_number'].isin([1,2,4,5,6,7,9,10])) &
                                    (transcript_user_summary['offer_received'] == 1) & 
                                    (transcript_user_summary['offer_viewed'] == 1) & 
                                    (transcript_user_summary['transaction'] == 1) &                                     
                                    (transcript_user_summary['offer_completed'] == 1),1,3)

In [ ]:
# this defines group 1 for informational promos
transcript_user_summary['group'] = np.where((transcript_user_summary['offer_number'].isin([3,8])) &
                                    (transcript_user_summary['offer_received'] == 1) & 
                                    (transcript_user_summary['offer_viewed'] == 1) & 
                                    (transcript_user_summary['transaction'] == 1) &                                     
                                    (transcript_user_summary['offer_completed'] == 0),1,3)

In [ ]:
# this defines group 2 for all promos
transcript_user_summary['group'] = np.where((transcript_user_summary['offer_number'].isin([1,2,3,4,5,6,7,8,9,10])) &
                                    (transcript_user_summary['offer_received'] == 1) & 
                                    (transcript_user_summary['offer_viewed'] == 1) & 
                                    (transcript_user_summary['transaction'] == 0) &                                     
                                    (transcript_user_summary['offer_completed'] == 0),2,3)

In [ ]:
# this defines group 4 for all promos
transcript_user_summary['group'] = np.where((transcript_user_summary['offer_number'].isin([1,2,3,4,5,6,7,8,9,10])) &
                                    (transcript_user_summary['offer_received'] == 1) & 
                                    (transcript_user_summary['offer_viewed'] == 0) & 
                                    (transcript_user_summary['transaction'] == 0) &                                     
                                    (transcript_user_summary['offer_completed'] == 0),4,3)

In [ ]:
transcript_sorted=transcript_sorted.merge(portfolio_clean,how='left',left_on='offer_number', right_on='offer_number')

In [ ]:
transcript_sorted.info()

In [ ]:
transcript_sorted.head()

In [ ]:
transcript_sorted.rename(columns={'reward_x':'reward_user', 'reward_y': 'reward_profile'},inplace=True)

In [ ]:
transcript_sorted.head()

### Issue

Change became_member_on to datetime

In [ ]:
# convert to datetime
profile_copy['became_member_on'] = pd.to_datetime(profile_copy.became_member_on)

### Test

In [ ]:
profile_copy.became_member_on

In [ ]:
transcript_profile.head()

The data above help understand the flow of an offer:<br>
1) Offer Received <br>
2) Offer Viewed <br>
3) Offer Completed (if required of the person)<br>
4) Transaction

In [ ]:
transcript_copy.tail(100)

In [ ]:
offer_keys = get_unique_values(transcript.value, type_dict=True)

In [ ]:
transcript_user_summary = transcript_copy.groupby(['user_id', 'offer_number']).agg({'offer_received': ['max'],
                                                                            'offer_viewed': ['max'],
                                                                             'transaction': ['sum'],
                                                                             'reward': ['sum'],                         
                                                                             'offer_completed': ['max']}).reset_index()

In [ ]:
transcript_user_summary

In [ ]:
portfolio_clean

In [ ]:
offer_charac[['offer_number', ]] = portfolio

In [ ]:
offer_transcript_user_summary.merge(portfolio_clean, left_on='offer_number', right_on='offer_number')

In [ ]:
offer_transcript.head()

In [ ]:
transcript_modified = extract_from_iterable_col(transcript, 'value', unique_keys=offer_keys)

Group 1 - 
* Offer is BOGO or Discount and user completes all components (offer received, offer viewed, transaction, offer completed)
* Offer is informational and user completes the following 3 components (offer received, offer viewed, transaction)

Group 2 - 
* Offer contains the following components only (offer received, offer viewed)

Group 4 - 
* Offer contains the following components only (offer received)

Group - 
* This is the most complicated.  The easiest way to solve this was to set the False value for all other groups to 3.  If they were not overwritten, group must equal 3.
         

In [ ]:
transcript_copy.sample(50)

In [ ]:
transcript_copy.head()

In [ ]:
transcript_modified.head(30)

In [ ]:
transcript_copy.head()

In [ ]:
df_combined = transcript_copy

## Analyze

## Visualize

## Model

### Data Preparation for modeling

### Predicting Model

## Evaluation